In [40]:
from dotenv import load_dotenv
load_dotenv()
import os
import openai
from pinecone import Pinecone, ServerlessSpec

In [39]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimensions=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

TypeError: Pinecone.create_index() got an unexpected keyword argument 'dimensions'

In [41]:
import json
from openai import OpenAI
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Thompson',
  'subject': 'Biology',
  'stars': 4,
  'review': "Dr. Thompson's lectures are engaging and informative. She makes complex topics easy to understand."},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Brilliant professor! His passion for coding is contagious. Challenging but rewarding classes.'},
 {'professor': 'Dr. Sarah Johnson',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Interesting material, but lectures can be a bit disorganized. Office hours are helpful.'},
 {'professor': 'Prof. David Williams',
  'subject': 'History',
  'stars': 4,
  'review': 'Prof. Williams brings history to life with his storytelling. Assignments are thought-provoking.'},
 {'professor': 'Dr. Lisa Rodriguez',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Excellent teacher! Dr. Rodriguez explains complex concepts clearly and is always willing to help.'},
 {'professor': 'Prof. Robert Taylor',
  'subject': 'Mat

In [22]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",

    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review['professor'],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

    

In [23]:
processed_data[0]

{'values': [0.028671574,
  -0.008983053,
  -0.0114871785,
  0.028671574,
  0.019503029,
  0.023265842,
  0.008532575,
  0.038290594,
  -0.00093076617,
  0.0042729126,
  0.021092951,
  0.018946558,
  -4.065374e-05,
  -0.024352288,
  0.0093010375,
  0.025756719,
  0.008956554,
  -0.036409188,
  0.06073498,
  0.060152005,
  0.048598584,
  -0.0123351365,
  0.025610976,
  -0.03171892,
  -0.040940464,
  -0.069638535,
  0.024816016,
  0.01890681,
  -0.029652026,
  -0.00913542,
  0.0724474,
  -0.021940907,
  -0.0055249752,
  -0.03672717,
  0.00040079255,
  0.033096854,
  -0.02252388,
  0.023345338,
  0.002403099,
  0.005117558,
  0.063861825,
  0.017515628,
  -0.038926564,
  0.012659745,
  0.018761067,
  -0.0091089215,
  -0.025663974,
  -0.014216542,
  0.024352288,
  0.03354733,
  -0.045153752,
  0.030817967,
  0.0039946763,
  -0.0101887435,
  -0.09603122,
  0.02933404,
  -0.0031102828,
  0.03680667,
  -0.034819268,
  -0.04523325,
  0.030764969,
  -0.018138347,
  0.005919143,
  0.0015394078,
 

In [26]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [42]:
index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}